In [ ]:
%pylab inline
rcParams['figure.figsize'] = (16, 4) #wide graphs by default

# Time Series

Some simple ready to use data sets: http://www.isds.duke.edu/~mw/ts_data_sets.html

In [ ]:
!wget http://www.isds.duke.edu/~mw/data-sets/ts_data/carinae.txt

In [ ]:
f = open('carinae.txt')

In [ ]:
values = [float(l) for l in f]

Some characters couldn't be converted! Need to try something else.

In [ ]:
f.seek(0)

values = []
for l in f:
    try:
        values.append(float(l))
    except:
        print 'line ignored: ', l

In [ ]:
plot(values)

In [ ]:
[(i, value) for i, value in enumerate(values[:10])]

Must clean some of the values which are invalid, but must keep the time series intact (i.e. leave the space)

In [ ]:
time_series = [(i, value) for i, value in enumerate(values) if value > 1]
time_series = array(time_series)
time_series.shape

In [ ]:
plot(*(time_series.T))

In [ ]:
plot(*(time_series.T), marker='o')
xlim((95,110))
grid()

Let's fill in the points instead.

In [ ]:
time_series = []

for i,value in enumerate(values):
    if value < 1:
        value = (values[i+1] - values[i-1])/2.0
    time_series.append(value)
    
time_series = array(time_series)
plot(time_series)
    

Oops! More than one consecutive point missing! We could do this ourselves or use a package like pandas.

http://pandas.pydata.org/

### LPC coeeficients from autoregression

In [ ]:
order = 24
win_size= 512
start = 0
signal = time_series[start:start+win_size]

In [ ]:
# Adapted from scikits.talkbox https://pypi.python.org/pypi/scikits.talkbox

from scipy.linalg import toeplitz

p = order + 1
r = zeros(p, signal.dtype)
# Number of non zero values in autocorrelation one needs for p LPC
# coefficients
nx = min([p, signal.size])
x = correlate(signal, signal, 'full')
r[:nx] = x[signal.size-1:signal.size+order]
phi = dot(inv(toeplitz(r[:-1])), -r[1:])

lpc = np.concatenate(([1.], phi))

In [ ]:
plot(lpc)

LPC coefficients are the feedback coefficients of an IIR filter. The LPC filter has no zeros.

In [ ]:
from scipy.signal import freqz

In [ ]:
h,w = freqz([1], lpc)
plot(h ,abs(w))

In [ ]:
from scipy.signal import tf2zpk
def PoleZeroPlot(b, a):
    (zeros,poles,gain) = tf2zpk(b, a)
    angle = np.linspace(-np.pi,np.pi,50)
    cirx = np.sin(angle)
    ciry = np.cos(angle)
    figure()
    plot(poles.real, poles.imag, 'x', zeros.real, zeros.imag, 'o', cirx,ciry, 'k-')
    grid()
    
    xlim((-2, 2))
    xlabel('Real')
    ylim((-1.5, 1.5))
    ylabel('Imag')
    gcf().set_figwidth(5)
    return (zeros,poles,gain)

In [ ]:
PoleZeroPlot([1], lpc);

In Linear Predictive Coding the coefficients calculated from auto regression are stored together with information about the excitation, like frequency and whether the frame is voiced (vowel/harmonic) or unvoiced (consonant/noisy).

Excellent LPC example:

http://nbviewer.ipython.org/github/jfsantos/ift6266h14/blob/master/notebooks/Speech%20representation%20examples.ipynb

## Compression

http://www.mp3-tech.org/programmer/frame_header.html

In [ ]:
f = open('noise.mp3')

In [ ]:
header = f.read(4)
header

In [ ]:
ord(header[0])

In [ ]:
hex(ord(header[0]))

In [ ]:
bin(ord(header[0]))

In [ ]:
for byte in header:
    print '%02x,'%ord(byte),

In [ ]:
bin(ord('c'))

In [ ]:
bin(ord('4'))

In [ ]:
frame = f.read(518)

In [ ]:
new_header = f.read(4)
new_header

In [ ]:
frame2 = f.read(518)

In [ ]:
new_header = f.read(4)
new_header

By: Andrés Cabrera mantaraya36@gmail.com
For MAT course MAT 201A at UCSB

This ipython notebook is licensed under the CC-BY-NC-SA license: http://creativecommons.org/licenses/by-nc-sa/4.0/

![http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png](http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png)